# Analyze predictions

In [15]:
import ast
import matplotlib.pyplot as plt
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from captum.attr import visualization as viz
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

FRAGE Lisanne: Feature names kommen nur einmal vor, oder? (auch wenn Token mehrfach im Satz vorkommt)
Was machen wir mit Attributions, die ja dann mehrere Werte haben je nachdem wie oft das Token vorkommt, einfach Mean nehmen, oder?


In [16]:
# convert strings into lists
def literal_eval_column(column):
    try:
        return ast.literal_eval(column)
    except (SyntaxError, ValueError):
        return column

In [19]:
# read distilbert predictions
dist_test = pd.read_csv('outputs/distilbert_attributions.csv')
dist_test = dist_test.applymap(literal_eval_column)

In [20]:
dist_test

,Unnamed: 0,index,attributions_pred,label_pred,score,tokens,label_gold,correct,sum_attributions_pred,mean_attributions_pred,attributions_gold,sum_attributions_gold,mean_attributions_gold
0,0,0,"[-0.011615987537462793, 0.003815996398574733, ...",rec.autos,0.801348,"[i, am, a, little, confused, on, all, of, the,...",rec.autos,True,2.790767,0.025371,"[-0.011615987537462793, 0.003815996398574733, ...",2.790767,0.025371
1,1,1,"[-0.32846125995080416, 0.021676696368081335, -...",comp.windows.x,0.915943,"[i, ', m, not, familiar, at, all, with, the, f...",comp.windows.x,True,3.255837,0.018395,"[-0.32846125995080416, 0.021676696368081335, -...",3.255837,0.018395
2,2,2,"[-0.014673215540331483, 0.1312478060786364, 0....",alt.atheism,0.072379,"[in, a, word, ,, yes, .]",alt.atheism,True,1.704919,0.284153,"[-0.014673215540331483, 0.1312478060786364, 0....",1.704919,0.284153
3,3,3,"[-0.6194980171999851, 0.0018658245313804055, 0...",talk.politics.mideast,0.808439,"[they, were, attacking, the, iraqi, ##s, to, d...",talk.politics.mideast,True,4.223599,0.008282,"[-0.6194980171999851, 0.0018658245313804055, 0...",4.223599,0.008282
4,4,4,"[-0.08235069488343288, 0.03930023488145007, -0...",alt.atheism,0.626613,"[i, ', ve, just, spent, two, solid, months, ar...",talk.religion.misc,False,1.368739,0.068437,"[-0.09691212604112631, 0.011566433822874374, -...",1.375414,0.068771
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7304,7304,8148,"[0.2254657002475486, 0.009065115322021058, -0....",sci.space,0.950029,"[henry, ,, if, i, read, you, correctly, ,, you...",sci.space,True,3.299645,0.013982,"[0.2254657002475486, 0.009065115322021058, -0....",3.299645,0.013982
7305,7305,8149,"[-0.10304464999994822, 0.03602233676625874, 0....",comp.sys.mac.hardware,0.768097,"[about, them, on, actually, ,, i, thought, mac...",comp.sys.mac.hardware,True,1.821632,0.046709,"[-0.10304464999994822, 0.03602233676625874, 0....",1.821632,0.046709
7306,7306,8150,"[-0.005376385427596206, 0.0014064797816725246,...",rec.sport.baseball,0.972486,"[i, sent, a, version, of, this, post, out, a, ...",rec.sport.baseball,True,1.739470,0.011079,"[-0.005376385427596206, 0.0014064797816725246,...",1.739470,0.011079
7307,7307,8151,"[0.09555778056771536, -0.021045926527705046, -...",sci.electronics,0.606990,"[i, have, this, kit, which, includes, the, fol...",misc.forsale,False,2.423564,0.009974,"[0.33790533123797367, 0.038921927909490554, -0...",4.352018,0.017910


In [6]:
# read SVM predictions
svm_test = pd.read_csv('outputs/coefs_test.csv')
svm_test = svm_test.applymap(literal_eval_column)

In [7]:
# exclude erroneous indices
with open('nohup-att-dist4.out', 'r', encoding='utf-8') as f:
    lines = f.readlines()
error_indices = [int(l.split(' ')[0]) for l in lines[7:] if l]

svm_test = svm_test[~svm_test.index.isin(error_indices)]
svm_test = svm_test.reset_index()

In [8]:
svm_test

,index,Unnamed: 0,true class no,true class name,pred class no,pred class name,feature ind,tfidf,feature names,coef true,coef pred,coef true*tfidf,coef pred*tfidf
0,0,0,7,rec.autos,4,comp.sys.mac.hardware,"[19173, 18937, 18270, 18198, 17388, 17145, 167...","[0.023220739480456238, 0.03033897195578605, 0....","[year, words, value, usually, time, tell, summ...","[-0.3644428172907367, -0.5038521269983025, -0....","[-0.4371961563133163, -0.43584702814775034, -0...","[-0.008462631715831708, -0.01528635555086465, ...","[-0.010152018047608341, -0.013223150763987291,..."
1,1,1,5,comp.windows.x,1,comp.graphics,"[18422, 18315, 17949, 16893, 15459, 15400, 135...","[0.02997111874877699, 0.04137978123401435, 0.0...","[view, ve, uncompressed, swamped, send, seeing...","[-0.3743281402679298, -0.36823803935498056, -0...","[-0.4741050755600308, -0.3507789408679207, -0....","[-0.011219033142978974, -0.015237609510551461,...","[-0.01420945951900757, -0.014515155834613814, ..."
2,2,2,0,alt.atheism,2,comp.os.ms-windows.misc,"[19184, 18931]","[0.4727116268736642, 0.5272883731263358]","[yes, word]","[-0.39628063158541726, -0.3763669255110641]","[-0.4034374258789668, -0.2686799939812319]","[-0.18732646205526576, -0.1984539038512898]","[-0.1907095619289697, -0.14167183691795746]"
3,3,3,17,talk.politics.mideast,0,alt.atheism,"[19195, 18958, 18912, 18734, 18598, 18315, 173...","[0.006670685520730314, 0.00481273683305861, 0....","[york, world, women, west, want, ve, times, ti...","[-0.29742243166061666, -0.3350764429380573, -0...","[-0.48446696985871884, -0.31526510095234483, -...","[-0.001984011508418877, -0.0016126347388182498...","[-0.003231726801108645, -0.001517287963531291,..."
4,4,4,19,talk.religion.misc,0,alt.atheism,"[18315, 17293, 16222, 16070, 12072, 11429, 114...","[0.07006597797491963, 0.07938683024646784, 0.1...","[ve, thing, spent, solid, objective, moral, mo...","[-0.44427465473197825, -0.5717712998446866, -0...","[-0.4059767700021195, -0.29545935499795983, -0...","[-0.031128538173265812, -0.0453911111205724, -...","[-0.02844515942529752, -0.023455581659953918, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7304,7527,7527,14,sci.space,14,sci.space,"[18607, 18295, 18294, 17756, 17294, 17279, 171...","[0.017691756398058683, 0.018650733635436213, 0...","[warm, varies, varied, try, things, thermal, t...","[-0.4585002959556874, -0.4052604615314347, -0....","[-0.4585002959556874, -0.4052604615314347, -0....","[-0.008111675544485832, -0.007558404920996732,...","[-0.008111675544485832, -0.007558404920996732,..."
7305,7528,7528,4,comp.sys.mac.hardware,4,comp.sys.mac.hardware,"[17316, 10666, 9725, 6231, 5108, 1304]","[0.14779138117244353, 0.2403323623065925, 0.20...","[thought, macs, jim, edu, day, actually]","[-0.4994580224796735, -0.15024876263583953, -0...","[-0.4994580224796735, -0.15024876263583953, -0...","[-0.0738155909799283, -0.036109640057913804, -...","[-0.0738155909799283, -0.036109640057913804, -..."
7306,7529,7529,9,rec.sport.baseball,9,rec.sport.baseball,"[18639, 18501, 18373, 17986, 17316, 17294, 156...","[0.020255864930211472, 0.029267291360587883, 0...","[wasn, void, version, understood, thought, thi...","[-0.40429532821330094, -0.3673492016941159, -0...","[-0.40429532821330094, -0.3673492016941159, -0...","[-0.00818935156020414, -0.010751316117061054, ...","[-0.00818935156020414, -0.010751316117061054, ..."
7307,7530,7530,6,misc.forsale,12,sci.electronics,"[18939, 18226, 18177, 18175, 17392, 16625, 166...","[0.01227474898275108, 0.028237249554259176, 0....","[work, uv, used, use, timer, stuff, student, r...","[-0.6091581217981357, -0.4379001261747615, -0....","[-0.31283400980582565, -0.1472211033427624, -0...","[-0.007477263035876224, -0.01236509514263832, ...","[-0.003839958943634, -0.004157119034742961, -0..."


In [9]:
assert len(svm_test) == len(dist_test)

In [28]:
# features (SVM) != tokens (DistilBERT)
def compare_pred(i):
    row_svm = svm_test.iloc[i]
    row_dist = dist_test.iloc[i]
    # initialize lists for common features and their attribution/coeff values
    feats = []
    val_svm_gold = []
    val_dist_gold = []
    val_svm_pred = []
    val_dist_pred = []
    for i, f in enumerate(row_svm['feature names']):
        if f in row_dist.tokens:
            feats.append(f)
            val_svm_gold.append(row_svm['coef true'][i])
            val_svm_pred.append(row_svm['coef pred'][i])
            # feature more than once in sentence
            if row_dist.tokens.count(f) > 1:
                indices = [j for j, token in enumerate(row_dist.tokens) if token == f]
                val_dist_gold.append(np.mean([row_dist['attributions_gold'][j] for j in indices]))
                val_dist_pred.append(np.mean([row_dist['attributions_pred'][j] for j in indices]))
            else:
                # feature index in distilbert tokens
                dist_index = row_dist.tokens.index(f)
                val_dist_gold.append(row_dist['attributions_gold'][dist_index])
                val_dist_pred.append(row_dist['attributions_pred'][dist_index])
    assert len(feats) == len(val_dist_gold)
    assert len(val_dist_gold) == len(val_dist_pred)
    assert len(val_dist_pred) == len(val_svm_gold)
    assert len(val_svm_gold) == len(val_svm_pred)
    print(feats)
    print(np.corrcoef(val_dist_gold, val_svm_gold))
    print(np.corrcoef(val_dist_pred, val_svm_pred))

In [29]:
compare_pred(0)

['year', 'words', 'value', 'usually', 'time', 'tell', 'summer', 'spring', 'se', 'performance', 'models', 'model', 'mid', 'little', 'le', 'heard', 'features', 'far', 'early', 'differences', 'demand', 'curious', 'confused', 'buy', 'book', 'best', '89', '88']
[[ 1.         -0.04194206]
 [-0.04194206  1.        ]]
[[1.         0.20352678]
 [0.20352678 1.        ]]


In [ ]:
# do some nice visualizations

In [14]:
# correlations
# compare predicted labels, have to ignore nan values with ma
np.corrcoef(np.ma.masked_invalid(svm_test['pred class name']),
            np.ma.masked_invalid(dist_test['label_pred']))

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''